# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'07-22-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'07-22-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-07-23 04:21:29,33.93911,67.709953,143183,6325,82586.0,54272.0,Afghanistan,367.811718,4.417424
1,NaN,NaN,NaN,Albania,2021-07-23 04:21:29,41.15330,20.168300,132797,2456,130109.0,232.0,Albania,4614.531934,1.849439
2,NaN,NaN,NaN,Algeria,2021-07-23 04:21:29,28.03390,1.659600,158213,4008,108537.0,45668.0,Algeria,360.796435,2.533294
3,NaN,NaN,NaN,Andorra,2021-07-23 04:21:29,42.50630,1.521800,14464,127,13988.0,349.0,Andorra,18719.989646,0.878042
4,NaN,NaN,NaN,Angola,2021-07-23 04:21:29,-11.20270,17.873900,41405,977,35082.0,5346.0,Angola,125.980230,2.359618


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,134653,136643,137853,137853,137853,137853,141489,142414,142414,143183
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132608,132616,132629,132647,132665,132686,132697,132740,132763,132797
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,147883,148797,149906,151103,152210,153309,154486,155784,157005,158213


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,5791,5923,5983,5983,5983,5983,6213,6266,6266,6325
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2456,2456,2456,2456,2456,2456,2456,2456,2456,2456
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3867,3882,3895,3910,3922,3938,3956,3979,3994,4008


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,130053,130059,130061,130067,130072,130074,130081,130086,130097,130109
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,102594,103192,103809,104397,104992,105604,106337,107041,107776,108537


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7306,7320,7329,7341,7346,7347,7350,7365,7375,7388
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,22337,22464,22556,22642,22734,22802,22876,23032,23184,23319
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2367,2373,2377,2386,2389,2394,2396,2411,2415,2418


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,113,113,113,113,113,113,113,113,113,113
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,318,321,323,323,323,323,323,324,324,324
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,60,60,60,60,60,60,60,60,60,60


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
656,1001.0,Autauga,Alabama,US,2021-07-23 04:21:29,32.539527,-86.644082,7388,113,NaN,NaN,"Autauga, Alabama, US",13223.791369,1.529507
693,1075.0,Lamar,Alabama,US,2021-07-23 04:21:29,33.779950,-88.096680,1491,38,NaN,NaN,"Lamar, Alabama, US",10800.434625,2.548625
694,1077.0,Lauderdale,Alabama,US,2021-07-23 04:21:29,34.901719,-87.656247,9883,253,NaN,NaN,"Lauderdale, Alabama, US",10657.938725,2.559951


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,34281864,610177,0.0
India,31257720,418987,30429339.0
Brazil,19523711,547016,17470332.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,34281864,610177,0.0,33671687.0,2021-07-23 04:21:29,37.950547,-91.419547
India,31257720,418987,30429339.0,409394.0,2021-07-23 04:21:29,23.088275,81.806127
Brazil,19523711,547016,17470332.0,1506363.0,2021-07-23 04:21:29,-12.669522,-48.480493
France,5996060,111749,410035.0,5474276.0,2021-07-23 04:21:29,6.430808,-34.730285
Russia,5979027,149012,5362424.0,467591.0,2021-07-23 04:21:29,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3893526,64235,0.0
Texas,3064743,52935,0.0
Florida,2450344,38388,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3893526,64235,0.0,3829291.0,2021-07-23 04:21:29,37.843962,-120.728594
Texas,3064743,52935,0.0,3011808.0,2021-07-23 04:21:29,31.660643,-98.653069
Florida,2450344,38388,0.0,2411956.0,2021-07-23 04:21:29,28.940755,-82.700744
New York,2131535,53798,0.0,2077737.0,2021-07-23 04:21:29,42.544151,-75.474183
Illinois,1406459,25831,0.0,1380628.0,2021-07-23 04:21:29,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1276254,24609,0.0
Arizona,Maricopa,572062,10403,0.0
Illinois,Cook,560675,10552,0.0
Florida,Miami-Dade,522734,6472,0.0
Texas,Harris,411095,6669,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1276254,24609,0.0,1251645.0,2021-07-23 04:21:29,34.308284,-118.228241,6037.0
Arizona,Maricopa,572062,10403,0.0,561659.0,2021-07-23 04:21:29,33.348359,-112.491815,4013.0
Illinois,Cook,560675,10552,0.0,550123.0,2021-07-23 04:21:29,41.841448,-87.816588,17031.0
Florida,Miami-Dade,522734,6472,0.0,516262.0,2021-07-23 04:21:29,25.611236,-80.551706,12086.0
Texas,Harris,411095,6669,0.0,404426.0,2021-07-23 04:21:29,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-20,142414,132740,155784,14379,41061,1275,4784219,227716,32266,654314,...,5542635,379376,120631,4,293866,65607,315761,6992,187602,88415
2021-07-21,142414,132763,157005,14379,41227,1275,4798851,227936,32426,654745,...,5586716,379613,121329,4,295746,71144,315876,6997,188573,91120
2021-07-22,143183,132797,158213,14464,41405,1277,4812351,228161,32588,655197,...,5626745,379911,122048,4,296863,78269,315876,7001,189731,93421


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-20,6266,2456,3979,127,970,42,102381,4572,915,10729,...,129109,5896,805,1,3392,334,3589,1371,3138,2747
2021-07-21,6266,2456,3994,127,973,42,102818,4573,915,10729,...,129182,5905,809,1,3426,370,3591,1371,3162,2809
2021-07-22,6325,2456,4008,127,977,42,103074,4575,916,10730,...,129266,5913,814,1,3443,370,3591,1372,3196,2870


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-20,82586,130086,107041,13930,34857,1225,4420995,218529,23965,639831,...,16686,369652,115514,3,276952,11443,310601,4162,174728,58155
2021-07-21,82586,130097,107776,13930,34893,1225,4431871,218585,23973,640018,...,17033,370213,116009,3,279177,11971,310961,4162,175429,59627
2021-07-22,82586,130109,108537,13988,35082,1225,4447953,218676,23985,640202,...,17290,370711,116755,3,280269,13421,310961,4162,177419,61723


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7306,7320,7329,7341,7346,7347,7350,7365,7375,7388
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1479,1481,1482,1482,1483,1485,1485,1486,1489,1491
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9720,9754,9767,9782,9793,9805,9813,9830,9872,9883
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,3150,3155,3160,3161,3162,3164,3164,3165,3170,3176
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,16454,16483,16513,16541,16561,16574,16587,16630,16666,16691


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-20,562311,72899,0,910436,367007,3875420,567215,351302,110457,49,...,876543,3049688,424730,24590,4249,686206,462577,165259,680743,63840
2021-07-21,563943,73374,0,911479,368466,3883186,567838,351530,110502,49,...,877850,3057874,425603,24609,4286,686884,463537,165398,681168,63988
2021-07-22,565510,73374,0,912653,370326,3893526,568597,351825,110573,49,...,879262,3064743,426418,24647,4301,687550,466099,165524,681562,64141


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-07-20        7365   23032    2411  2747   7108    1252   2315   14948   
2021-07-21        7375   23184    2415  2753   7133    1253   2328   14978   
2021-07-22        7388   23319    2418  2761   7147    1253   2335   15030   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-20         3801     1911  ...    2932    722     3288      829   
2021-07-21         3805     1920  ...    2936    724     3289      831   
2021-07-22         3813     1923  ...    2938    732     3290      833   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-20           4906  3848  2351          0      935    669  
2021-07-21           4917  3853  2361          0      937    669  
2021-07-22           4930  3865  2368          0      937    670  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-20,11460,381,0,18117,6007,64144,6889,8282,1697,0,...,12638,52865,2420,259,33,11487,6056,2919,8223,766
2021-07-21,11462,382,0,18127,6020,64163,6895,8282,1697,0,...,12642,52904,2424,259,33,11491,6063,2920,8229,766
2021-07-22,11468,382,0,18137,6035,64235,6901,8286,1697,0,...,12657,52935,2425,259,35,11493,6066,2934,8234,766


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-07-20         113     324      60   65    139      42     71     332   
2021-07-21         113     324      60   65    139      42     71     332   
2021-07-22         113     324      60   65    139      42     71     332   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-20          124       47  ...      33     14       31        7   
2021-07-21          124       47  ...      33     14       31        7   
2021-07-22          124       47  ...      33     14       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-20             43    11    13          0       26      6  
2021-07-21             43    11    13          0       26      6  
2021-07-22             43    11    13          0       26      6  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-20,0.006538,0.000324,0.008402,0.001393,0.003789,0.000000,0.003161,0.000853,0.004545,0.000483,...,0.008495,0.000802,0.005485,0.0,0.000000,0.090179,0.000000,0.000716,0.007102,0.031295
2021-07-21,0.000000,0.000173,0.007838,0.000000,0.004043,0.000000,0.003058,0.000966,0.004959,0.000659,...,0.007953,0.000625,0.005786,0.0,0.006397,0.084396,0.000364,0.000715,0.005176,0.030594
2021-07-22,0.005400,0.000256,0.007694,0.005911,0.004318,0.001569,0.002813,0.000987,0.004996,0.000690,...,0.007165,0.000785,0.005926,0.0,0.003777,0.100149,0.000000,0.000572,0.006141,0.025252


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-20,0.008531,0.0,0.005814,0.0,0.001032,0.0,0.004178,0.000657,0.000000,0.000000,...,0.000791,0.001189,0.004994,0.0,0.000000,0.000000,0.000000,0.000730,0.008031,0.018539
2021-07-21,0.000000,0.0,0.003770,0.0,0.003093,0.0,0.004268,0.000219,0.000000,0.000000,...,0.000565,0.001526,0.004969,0.0,0.010024,0.107784,0.000557,0.000000,0.007648,0.022570
2021-07-22,0.009416,0.0,0.003505,0.0,0.004111,0.0,0.002490,0.000437,0.001093,0.000093,...,0.000650,0.001355,0.006180,0.0,0.004962,0.000000,0.000000,0.000729,0.010753,0.021716


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-20,0.0,0.000038,0.006620,0.002375,0.001926,0.0,0.002990,0.000467,0.000250,0.000205,...,0.012070,0.001493,0.005650,0.0,0.000000,0.035847,0.000000,0.0,0.008124,0.043813
2021-07-21,0.0,0.000085,0.006867,0.000000,0.001033,0.0,0.002460,0.000256,0.000334,0.000292,...,0.020796,0.001518,0.004285,0.0,0.008034,0.046142,0.001159,0.0,0.004012,0.025312
2021-07-22,0.0,0.000092,0.007061,0.004164,0.005417,0.0,0.003629,0.000416,0.000501,0.000287,...,0.015088,0.001345,0.006431,0.0,0.003911,0.121126,0.000000,0.0,0.011344,0.035152


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-20,0.002480,0.000000,NaN,0.001269,0.005135,0.002526,0.000962,0.000621,0.000308,0.0,...,0.001045,0.001845,0.001318,0.000448,0.008066,0.001052,0.001581,0.000496,0.000722,0.002056
2021-07-21,0.002902,0.006516,NaN,0.001146,0.003975,0.002004,0.001098,0.000649,0.000407,0.0,...,0.001491,0.002684,0.002055,0.000773,0.008708,0.000988,0.002075,0.000841,0.000624,0.002318
2021-07-22,0.002779,0.000000,NaN,0.001288,0.005048,0.002663,0.001337,0.000839,0.000643,0.0,...,0.001608,0.002246,0.001915,0.001544,0.003500,0.000970,0.005527,0.000762,0.000578,0.002391


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-07-20      0.002041  0.006819  0.006260  0.005123  0.000704  0.000000   
2021-07-21      0.001358  0.006600  0.001659  0.002184  0.003517  0.000799   
2021-07-22      0.001763  0.005823  0.001242  0.002906  0.001963  0.000000   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-07-20      0.005211  0.001340  0.003167  0.001048  ...  0.000683   
2021-07-21      0.005616  0.002007  0.001052  0.004710  ...  0.001364   
2021-07-22      0.003007  0.003472  0.002102  0.001562  ...  0.000681   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-07-20      0.001387  0.001218  0.004848   0.003067  0.001562  0.005130   
2021-07-21      0.002770  0.000304  0.002413   0.002242  0.001299  0.004254   
2021-07-22      0.011050  0.000304  0.002407   0.002644  0.003114  0.002965   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-07-20            NaN  0.000000  0.006015  
2021-07-21            NaN  0.002139  0.000000  
2021-07-22            NaN  0.000000  0.001495  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-20,0.001486,0.000000,NaN,0.000000,0.000000,0.000172,0.000436,0.000000,0.0,NaN,...,0.000633,0.000492,0.001656,0.0,0.031250,0.000348,0.002151,0.000000,0.001340,0.007895
2021-07-21,0.000175,0.002625,NaN,0.000552,0.002164,0.000296,0.000871,0.000000,0.0,NaN,...,0.000317,0.000738,0.001653,0.0,0.000000,0.000348,0.001156,0.000343,0.000730,0.000000
2021-07-22,0.000523,0.000000,NaN,0.000552,0.002492,0.001122,0.000870,0.000483,0.0,NaN,...,0.001187,0.000586,0.000413,0.0,0.060606,0.000174,0.000495,0.004795,0.000608,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                         \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock Butler   Calhoun   
2021-07-20         0.0  0.003096     0.0  0.0    0.0     0.0    0.0  0.003021   
2021-07-21         0.0  0.000000     0.0  0.0    0.0     0.0    0.0  0.000000   
2021-07-22         0.0  0.000000     0.0  0.0    0.0     0.0    0.0  0.000000   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-20          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2021-07-21          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2021-07-22          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-20        0.04878   0.0   0.0        NaN      0.0    0.0  
2021-07-21        0.00000   0.0   0.0        NaN      0.0    0.0  
2021-07-22        0.00000   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-20,0.010168,0.000218,0.007942,0.002372,0.003142,0.001411,0.002828,0.000697,0.004041,0.000473,...,0.008417,0.000686,0.005023,1.346323e-28,0.001897,0.088565,0.000132,0.000729,0.006010,0.026631
2021-07-21,0.005084,0.000196,0.007890,0.001186,0.003592,0.000706,0.002943,0.000831,0.004500,0.000566,...,0.008185,0.000655,0.005404,6.731613e-29,0.004147,0.086481,0.000248,0.000722,0.005593,0.028613
2021-07-22,0.005242,0.000226,0.007792,0.003549,0.003955,0.001137,0.002878,0.000909,0.004748,0.000628,...,0.007675,0.000720,0.005665,3.365807e-29,0.003962,0.093315,0.000124,0.000647,0.005867,0.026933


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-20,0.014300,1.290479e-11,0.004985,1.367830e-26,0.004357,7.604046e-21,0.003693,0.000735,0.000379,0.000018,...,0.000507,0.001109,0.005163,0.0,0.002113,0.106146,0.000182,0.000822,0.009081,0.022004
2021-07-21,0.007150,6.452395e-12,0.004377,6.839151e-27,0.003725,3.802023e-21,0.003980,0.000477,0.000189,0.000009,...,0.000536,0.001318,0.005066,0.0,0.006068,0.106965,0.000369,0.000411,0.008365,0.022287
2021-07-22,0.008283,3.226197e-12,0.003941,3.419575e-27,0.003918,1.901012e-21,0.003235,0.000457,0.000641,0.000051,...,0.000593,0.001336,0.005623,0.0,0.005515,0.053483,0.000185,0.000570,0.009559,0.022001


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-20,0.000058,0.000039,0.006490,0.002221,0.001463,0.000255,0.003161,0.000457,0.000349,0.000198,...,0.007964,0.001569,0.005101,3.231174e-27,0.002216,0.034322,0.000113,0.000328,0.009254,0.038556
2021-07-21,0.000029,0.000062,0.006678,0.001110,0.001248,0.000128,0.002810,0.000357,0.000341,0.000245,...,0.014380,0.001544,0.004693,1.615587e-27,0.005125,0.040232,0.000636,0.000164,0.006633,0.031934
2021-07-22,0.000014,0.000077,0.006869,0.002637,0.003332,0.000064,0.003220,0.000387,0.000421,0.000266,...,0.014734,0.001444,0.005562,8.077936e-28,0.004518,0.080679,0.000318,0.000082,0.008988,0.033543


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-20,0.001929,0.001741,NaN,0.001294,0.004543,0.001784,0.000872,0.000627,0.000354,5.324689e-149,...,0.001208,0.001347,0.001674,0.000564,0.007970,0.000924,0.001865,0.000650,0.000538,0.001848
2021-07-21,0.002416,0.004128,NaN,0.001220,0.004259,0.001894,0.000985,0.000638,0.000381,2.662345e-149,...,0.001350,0.002016,0.001865,0.000668,0.008339,0.000956,0.001970,0.000746,0.000581,0.002083
2021-07-22,0.002597,0.002064,NaN,0.001254,0.004654,0.002278,0.001161,0.000739,0.000512,1.331172e-149,...,0.001479,0.002131,0.001890,0.001106,0.005919,0.000963,0.003749,0.000754,0.000580,0.002237


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-07-20      0.001270  0.005085  0.003847  0.003545  0.000907  0.000062   
2021-07-21      0.001314  0.005842  0.002753  0.002864  0.002212  0.000431   
2021-07-22      0.001538  0.005833  0.001998  0.002885  0.002087  0.000215   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-07-20      0.003111  0.001142  0.001974  0.000838  ...  0.000772   
2021-07-21      0.004363  0.001574  0.001513  0.002774  ...  0.001068   
2021-07-22      0.003685  0.002523  0.001808  0.002168  ...  0.000875   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-07-20      0.002680  0.000701  0.004478   0.002085  0.001275  0.003036   
2021-07-21      0.002725  0.000503  0.003445   0.002164  0.001287  0.003645   
2021-07-22      0.006887  0.000403  0.002926   0.002404  0.002201  0.003305   

Province_State                                
Admin2         Unassigned Washakie    Weston  
2021-07-20           -1.0  0.00062  0.003385  
2021-07-21           -1.0  0.00138  0.001692  
2021-07-22           -1.0  0.00069  0.001594  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-20,0.000790,0.000012,NaN,0.000167,0.000716,0.000282,0.000331,0.000006,0.000023,NaN,...,0.001112,0.000395,0.001227,0.000969,0.015629,0.000336,0.001194,0.000046,0.000817,0.003994
2021-07-21,0.000482,0.001318,NaN,0.000360,0.001440,0.000289,0.000601,0.000003,0.000012,NaN,...,0.000714,0.000566,0.001440,0.000485,0.007815,0.000342,0.001175,0.000194,0.000773,0.001997
2021-07-22,0.000503,0.000659,NaN,0.000456,0.001966,0.000706,0.000736,0.000243,0.000006,NaN,...,0.000950,0.000576,0.000926,0.000242,0.034210,0.000258,0.000835,0.002494,0.000690,0.000999


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                        \
Admin2               Autauga   Baldwin       Barbour          Bibb   
2021-07-20      5.796277e-16  0.001722  6.314051e-11  9.536743e-07   
2021-07-21      2.898138e-16  0.000861  3.157026e-11  4.768372e-07   
2021-07-22      1.449069e-16  0.000430  1.578513e-11  2.384186e-07   

Province_State                                                                \
Admin2                Blount       Bullock        Butler   Calhoun  Chambers   
2021-07-20      4.970621e-23  1.652851e-22  4.090787e-19  0.001511  0.000064   
2021-07-21      2.485311e-23  8.264253e-23  2.045393e-19  0.000755  0.000032   
2021-07-22      1.242655e-23  4.132127e-23  1.022697e-19  0.000378  0.000016   

Province_State            ...       Wyoming                          \
Admin2          Cherokee  ...          Park    Platte      Sheridan   
2021-07-20      0.000011  ...  9.695689e-07  0.000301  1.959556e-40   
2021-07-21      0.000005  ...  4.847844e-07  0.000150  9.797779e-41   
2021-07-22      0.000003  ...  2.423922e-07  0.000075  4.898890e-41   

Province_State                                                       \
Admin2              Sublette Sweetwater         Teton         Uinta   
2021-07-20      8.608505e-61   0.024488  5.052748e-14  2.757269e-25   
2021-07-21      4.304253e-61   0.012244  2.526374e-14  1.378634e-25   
2021-07-22      2.152126e-61   0.006122  1.263187e-14  6.893172e-26   

Province_State                                         
Admin2         Unassigned      Washakie        Weston  
2021-07-20       0.752941  5.348609e-53  4.547474e-14  
2021-07-21       0.752941  2.674305e-53  2.273737e-14  
2021-07-22       0.752941  1.337152e-53  1.136868e-14  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
